# 네거티브 샘플링을 이용한 Word2Vec 구현(Skip-Gram with Negative Sampling, SGNS)

reference https://wikidocs.net/69141

20뉴스그룹 데이터 전처리하기

In [21]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kwon_notebook\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


20뉴스그룹 데이터를 사용함. 하나의 샘플에 최소 단어 2개가 있어야함. 그래야지 중심 단어, 주변 단어의 관계가 성립하며 그렇지 않으면 샘플을 구성할 수 없어 에러가 발생함 전처리 과정에서 이를 만족하지 않는 샘플들을 제거 하겠음

In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('총 샘플 수 :',len(documents))

총 샘플 수 : 11314


In [5]:
news_df = pd.DataFrame({'document':documents})
news_df['document']

0        Well i'm not sure about the story nad it did s...
1        \n\n\n\n\n\n\nYeah, do you expect people to re...
2        Although I realize that principle is not one o...
3        Notwithstanding all the legitimate fuss about ...
4        Well, I will have to change the scoring on my ...
                               ...                        
11309    Danny Rubenstein, an Israeli journalist, will ...
11310                                                   \n
11311    \nI agree.  Home runs off Clemens are always m...
11312    I used HP DeskJet with Orange Micros Grappler ...
11313                                          ^^^^^^\n...
Name: document, Length: 11314, dtype: object

In [10]:
news_df = pd.DataFrame({'document':documents})
# 특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

C:\Users\kwon_notebook\anaconda3\envs\tensorNN\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
news_df['clean_doc'][0]

'well sure about story seem biased what disagree with your statement that media ruin israels reputation that rediculous media most israeli media world having lived europe realize that incidences such described letter have occured media whole seem ignore them subsidizing israels existance europeans least same degree think that might reason they report more clearly atrocities what shame that austria daily reports inhuman acts commited israeli soldiers blessing received from government makes some holocaust guilt away after look jews treating other races when they power unfortunate'

데이터 프레임에 null 값이 있는지 확인

In [12]:
news_df.isnull().values.any()

False

null 값이 없지만, 빈 값 유무도 확인해야함. 모든 빈값을 null값으로 변환하고 다시 null 값이 있는지 확인

In [13]:
news_df.replace("", float("NaN"), inplace=True)
news_df.isnull().values.any()

True

In [14]:
news_df.dropna(inplace=True)
print('총 샘플 수 :', len(news_df))

총 샘플 수 : 10995


NLTK 에서 정의한 불용어 리스트를 사용하여 불용어를 제거

In [22]:
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()

In [27]:
tokenized_doc[0]

['well',
 'sure',
 'story',
 'seem',
 'biased',
 'disagree',
 'statement',
 'media',
 'ruin',
 'israels',
 'reputation',
 'rediculous',
 'media',
 'israeli',
 'media',
 'world',
 'lived',
 'europe',
 'realize',
 'incidences',
 'described',
 'letter',
 'occured',
 'media',
 'whole',
 'seem',
 'ignore',
 'subsidizing',
 'israels',
 'existance',
 'europeans',
 'least',
 'degree',
 'think',
 'might',
 'reason',
 'report',
 'clearly',
 'atrocities',
 'shame',
 'austria',
 'daily',
 'reports',
 'inhuman',
 'acts',
 'commited',
 'israeli',
 'soldiers',
 'blessing',
 'received',
 'government',
 'makes',
 'holocaust',
 'guilt',
 'away',
 'look',
 'jews',
 'treating',
 'races',
 'power',
 'unfortunate']

모든 샘플 중 단어가 1개 이하인 경우를 모두 찾아 제거하면

In [28]:
drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence) <=1]
tokenized_doc = np.delete(tokenized_doc, drop_train, axis=0)
print('총 샘플 수 :', len(tokenized_doc))

총 샘플 수 : 10940


C:\Users\kwon_notebook\anaconda3\envs\tensorNN\lib\site-packages\numpy\lib\function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


단어 집합을 생성하고, 정수 인코딩을 생성

In [30]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

word2idx = tokenizer.word_index
idx2word = {value : key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)

encoded[:2]

[[9,
  59,
  603,
  207,
  3278,
  1495,
  474,
  702,
  9470,
  13686,
  5533,
  15227,
  702,
  442,
  702,
  70,
  1148,
  1095,
  1036,
  20294,
  984,
  705,
  4294,
  702,
  217,
  207,
  1979,
  15228,
  13686,
  4865,
  4520,
  87,
  1530,
  6,
  52,
  149,
  581,
  661,
  4406,
  4988,
  4866,
  1920,
  755,
  10668,
  1102,
  7837,
  442,
  957,
  10669,
  634,
  51,
  228,
  2669,
  4989,
  178,
  66,
  222,
  4521,
  6066,
  68,
  4295],
 [1026,
  532,
  2,
  60,
  98,
  582,
  107,
  800,
  23,
  79,
  4522,
  333,
  7838,
  864,
  421,
  3825,
  458,
  6488,
  458,
  2700,
  4730,
  333,
  23,
  9,
  4731,
  7262,
  186,
  310,
  146,
  170,
  642,
  1260,
  107,
  33568,
  13,
  985,
  33569,
  33570,
  9471,
  11491]]

In [31]:
vocab_size = len(word2idx) + 1
print('단어 집합의 크기 : ', vocab_size)

단어 집합의 크기 :  64277


# 네거티브 샘플링을 통한 데이터셋 구성하기

네거티브 샘플링을 위해서 케라스에서 제공하는 전처리 도구인 skipgrams를 사용함. 어떤 전처리가 수행되는지 그 결과를 확인하기 위해서 상위 10개의 뉴스그룹 샘플에 대해서만 수행

In [36]:
from tensorflow.keras.preprocessing.sequence import skipgrams

skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:10]]

In [37]:
# 첫번재 샘플인 skip_grams[0] 내 skipgrams 로 형성된 데이터셋 확인
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(5):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(idx2word[pairs[i][0]], pairs[i][0],                           idx2word[pairs[i][1]], pairs[i][1],
      labels[i]))

(reports (755), situations (2195)) -> 0
(ruin (9470), rediculous (15227)) -> 1
(israeli (442), media (702)) -> 1
(soldiers (957), sorcerors (51146)) -> 0
(biased (3278), story (603)) -> 1


윈도우 크기 내에서 중심단어, 주변단어의 관계를 가지는 경우에는 1의 레이블을 갖도록 하고, 그렇지 않은 경우는 0의 레이블을 가지도록 하여 데이터셋을 구성함.

In [38]:
print('전체 샘플 수 :',  len(skip_grams))

전체 샘플 수 : 10


encoded 중 상위 10개의 뉴스 그룹 샘플에 대해서만 수행하였으므로 10이 출력됨. 그리고 10개의 뉴스그룹 샘플 각각은 수많은 중심 단어, 주변단어의 쌍으로 된 샘플들을 갖고 있음. 첫번재 뉴스그룹 샘플이 가지고 있는 pairs와 labels의 개수를 출력하면

In [40]:
print(len(pairs))
print(len(labels))

2220
2220


위 작업을 모든 뉴스그룹 샘플에 대해서 수행하면

In [42]:
from tqdm import tqdm
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in tqdm(encoded)]

100%|██████████| 10940/10940 [02:33<00:00, 71.23it/s] 


# Skip-Gram with Negative Sampling(SGNS) 구현하기

In [43]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

embedding_dim = 100

# 중심 단어를 위한 임베딩 테이블
# 임베딩 레이어에 관한 레퍼런스 https://wikidocs.net/32105
w_inputs = Input(shape=(1,), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding = Embedding(vocab_size, embedding_dim)(c_inputs)

각 임베딩 테이블은 중심 단어와 주변 단어 각각을 위한 임베딩 테이블이며 각 단어는 임베딩 테이블을 거쳐서 내적을 수행하고, 내적의 결과는 1 또는 0을 예측하기 위해서 시그모이드 함수를 활성화 함수로 거쳐 최종 예측값을 얻음

In [47]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(
    loss='binary_crossentropy',
    optimizer='adam'
)
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 100)       6427700     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 100)       6427700     input_2[0][0]                    
_______________________________________________________________________________________

In [ ]:
for epoch in range(1, 6):
    loss = 0
    for _, elem in tqdm(enumerate(skip_grams)):
        first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        x = [first_elem, second_elem]
        y = labels
        loss += model.train_on_batch(x, y)
    print('Epoch :', epoch, "Loss :", loss)